In [118]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from seaborn import scatterplot
from seaborn import regplot
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor

Carga del Modelo Anterior

In [119]:
ModeloParo = pd.read_csv('CombinadoFinalParaCovid.csv',sep=';', encoding="latin-1")


Agrupamiento por periodo

In [120]:
Agrupado = pd.DataFrame(ModeloParo.groupby(by=['Periodo'], as_index=False)['num_casos'].sum())
Agrupado

,Periodo,num_casos
0,202001,78257
1,202002,92144
2,202003,572036
3,202004,1177854
4,202101,1319036
5,202102,534329
6,202103,1108678
7,202104,1825734
8,202201,1727120
9,202202,10549


In [121]:
dataParoPredic = pd.read_csv('DatosParoPrediccion.csv',sep=';', encoding="latin-1")
dataParoPredic['Periodo'] = dataParoPredic['Periodo'].str.replace('T','0')
dataParoPredic['Periodo'] = pd.to_numeric(dataParoPredic['Periodo'], downcast='integer')
dataParoPredic = pd.merge(dataParoPredic,Agrupado, on="Periodo", how="left")
dataParoPredic = dataParoPredic.sort_values(by=['Periodo', 'Sexo', 'Edad'])
dataParoPredic.fillna(0, inplace=True)
dataParoPredic['Periodo'] = pd.to_numeric(dataParoPredic['Periodo'], downcast='integer')
dataParoPredic['num_casos'] = pd.to_numeric(dataParoPredic['num_casos'], downcast='integer')
dataParoPredic = dataParoPredic.drop(dataParoPredic[dataParoPredic['Total']=='..'].index)
dataParoPredic['Total'] = pd.to_numeric(dataParoPredic['Total'])
dataParoPredic

,Edad,Sexo,Sector económico,Periodo,Total,num_casos
120,55 y más años,Hombres,Agricultura,202201,0,1727120
124,55 y más años,Hombres,Industria,202201,0,1727120
128,55 y más años,Hombres,Construcción,202201,0,1727120
132,55 y más años,Hombres,Servicios,202201,0,1727120
136,55 y más años,Hombres,Parados que buscan primer empleo o han dejado ...,202201,0,1727120
...,...,...,...,...,...,...
263,De 25 a 54 años,Mujeres,Agricultura,202304,0,3742
267,De 25 a 54 años,Mujeres,Industria,202304,0,3742
271,De 25 a 54 años,Mujeres,Construcción,202304,0,3742
275,De 25 a 54 años,Mujeres,Servicios,202304,0,3742


In [146]:
dataParo = pd.read_csv('DatosParo.csv',sep=';', encoding="latin-1")
dataParo['Periodo'] = dataParo['Periodo'].str.replace('T','0')
dataParo['Periodo'] = pd.to_numeric(dataParo['Periodo'], downcast='integer')
dataParo = pd.merge(dataParo,Agrupado, on="Periodo", how="left")
dataParo = dataParo.sort_values(by=['Periodo', 'Sexo', 'Edad'])
dataParo.fillna(0, inplace=True)
dataParo['Periodo'] = pd.to_numeric(dataParo['Periodo'], downcast='integer')
dataParo['num_casos'] = pd.to_numeric(dataParo['num_casos'], downcast='integer')
dataParo = dataParo.drop(dataParo[dataParo['Total']=='..'].index)
dataParo['Total'] = pd.to_numeric(dataParo['Total'].str.replace(',','.'))
dataParo = dataParo.sort_values(['Sexo','Edad','Sector económico','Periodo'])
dataParo['lag1'] = dataParo.groupby(['Sexo','Edad','Sector económico'])['Total'].shift(periods=1, fill_value=0.0)
dataParo['lag2'] = dataParo.groupby(['Sexo','Edad','Sector económico'])['Total'].shift(2, fill_value=0.0)
dataParo['lag3'] = dataParo.groupby(['Sexo','Edad','Sector económico'])['Total'].shift(3, fill_value=0.0)
dataParo['lag4'] = dataParo.groupby(['Sexo','Edad','Sector económico'])['Total'].shift(4, fill_value=0.0)
dataParo

,Edad,Sexo,Sector económico,Periodo,Total,num_casos,lag1,lag2,lag3,lag4
371,55 y más años,Hombres,Agricultura,201901,11.4,0,0.0,0.0,0.0,0.0
370,55 y más años,Hombres,Agricultura,201902,17.8,0,11.4,0.0,0.0,0.0
369,55 y más años,Hombres,Agricultura,201903,17.5,0,17.8,11.4,0.0,0.0
368,55 y más años,Hombres,Agricultura,201904,18.1,0,17.5,17.8,11.4,0.0
367,55 y más años,Hombres,Agricultura,202001,17.0,78257,18.1,17.5,17.8,11.4
...,...,...,...,...,...,...,...,...,...,...
340,De 25 a 54 años,Mujeres,Servicios,202004,641.7,1177854,680.7,672.9,567.7,500.4
339,De 25 a 54 años,Mujeres,Servicios,202101,591.8,1319036,641.7,680.7,672.9,567.7
338,De 25 a 54 años,Mujeres,Servicios,202102,452.5,534329,591.8,641.7,680.7,672.9
337,De 25 a 54 años,Mujeres,Servicios,202103,465.2,1108678,452.5,591.8,641.7,680.7


Escribir a 1 - 0 dummies

In [152]:
dataParoDummies = pd.DataFrame()
dataParoDummies =pd.get_dummies(dataParo[['Edad','Sexo','Sector económico']])
dataParoConDummies = dataParo.join(dataParoDummies,lsuffix='_left', rsuffix='_right')

dataParoConDummies = dataParoConDummies[['Periodo', 'Total', 'num_casos',
       'Edad_55 y más años', 'Edad_De 16 a 19 años', 'Edad_De 20 a 24 años',
       'Edad_De 25 a 54 años', 'Sexo_Hombres', 'Sexo_Mujeres',
       'Sector económico_Agricultura', 'Sector económico_Construcción',
       'Sector económico_Industria',
       'Sector económico_Parados que buscan primer empleo o han dejado su último empleo hace más de 1 año',
       'Sector económico_Servicios']]

In [153]:
dataParoDummiesPredic = pd.DataFrame()
dataParoDummiesPredic =pd.get_dummies(dataParoPredic[['Edad','Sexo','Sector económico']])
dataParoDummiesPredic = dataParoPredic.join(dataParoDummiesPredic,lsuffix='_left', rsuffix='_right')

dataParoDummiesPredic = dataParoDummiesPredic[['Periodo', 'Total', 'num_casos',
       'Edad_55 y más años', 'Edad_De 16 a 19 años', 'Edad_De 20 a 24 años',
       'Edad_De 25 a 54 años', 'Sexo_Hombres', 'Sexo_Mujeres',
       'Sector económico_Agricultura', 'Sector económico_Construcción',
       'Sector económico_Industria',
       'Sector económico_Parados que buscan primer empleo o han dejado su último empleo hace más de 1 año',
       'Sector económico_Servicios']]

In [155]:
X = dataParoConDummies[['Periodo', 'num_casos',
       'Edad_55 y más años', 'Edad_De 16 a 19 años', 'Edad_De 20 a 24 años',
       'Edad_De 25 a 54 años', 'Sexo_Hombres', 'Sexo_Mujeres',
       'Sector económico_Agricultura', 'Sector económico_Construcción',
       'Sector económico_Industria',
       'Sector económico_Parados que buscan primer empleo o han dejado su último empleo hace más de 1 año',
       'Sector económico_Servicios']]
y = dataParoConDummies[['Total']]

X_predict = dataParoDummiesPredic[['Periodo', 'num_casos',
       'Edad_55 y más años', 'Edad_De 16 a 19 años', 'Edad_De 20 a 24 años',
       'Edad_De 25 a 54 años', 'Sexo_Hombres', 'Sexo_Mujeres',
       'Sector económico_Agricultura', 'Sector económico_Construcción',
       'Sector económico_Industria',
       'Sector económico_Parados que buscan primer empleo o han dejado su último empleo hace más de 1 año',
       'Sector económico_Servicios']]
y_predict = dataParoDummiesPredic[['Total']]

In [150]:
LinearAccuracyFinal = 0
tscv = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
for train_index, test_index in tscv.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    reg = LinearRegression()
    reg.fit(X_train,y_train)
    LinearAccuracy = reg.score(X_test,y_test)
    if(LinearAccuracy > LinearAccuracyFinal):
       LinearAccuracyFinal = LinearAccuracy

print('Final Accuracy: ',LinearAccuracyFinal)

Final Accuracy:  0.9548435618347162


In [98]:
AccurateMaxDepth = 0
RandomAccuracyFinal = 0
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    for maxDepths in range(1,11):
        RandomModel = RandomForestRegressor (n_estimators = 500, max_depth=maxDepths, random_state=0)
        RandomModel.fit(X_train,y_train.values.ravel())
        RandomForestAccuracy = RandomModel.score(X_test,y_test)
        if(RandomForestAccuracy>RandomAccuracyFinal):
            RandomAccuracyFinal = RandomForestAccuracy
            AccurateMaxDepth = maxDepths

print('Max Depth: ',AccurateMaxDepth)
print('Final Accuracy: ',RandomAccuracyFinal)

Max Depth:  7
Final Accuracy:  0.9922322668643275


In [99]:
BoostAccurateMaxDepth = 0
BoostAccuracyFinal = 0
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    for maxDepths in range(1,11):
       BoostModel = XGBRegressor(n_estimators = 1000, max_depth=maxDepths, random_state=0)
       BoostModel.fit(X_train,y_train)
       BoostAccuracy = BoostModel.score(X_test,y_test)
       if(BoostAccuracy>BoostAccuracyFinal):
            BoostAccuracyFinal = BoostAccuracy
            BoostAccurateMaxDepth = maxDepths

print('Max Depth: ',BoostAccurateMaxDepth)
print('Final Accuracy: ',BoostAccuracyFinal)    

Max Depth:  3
Final Accuracy:  0.9979171930552337


Entrenamiento Final

In [156]:
 BoostModelFinal = RandomForestRegressor(n_estimators = 1000, max_depth=3, random_state=0)
 BoostModelFinal.fit(X,y.values.ravel())
 predictions = BoostModelFinal.predict(X_predict)
 dataParoDummiesPredic['Total'] = predictions
dataParoDummiesPredic

,Periodo,Total,num_casos,Edad_55 y más años,Edad_De 16 a 19 años,Edad_De 20 a 24 años,Edad_De 25 a 54 años,Sexo_Hombres,Sexo_Mujeres,Sector económico_Agricultura,Sector económico_Construcción,Sector económico_Industria,Sector económico_Parados que buscan primer empleo o han dejado su último empleo hace más de 1 año,Sector económico_Servicios
120,202201,7.254880,1727120,1,0,0,0,1,0,1,0,0,0,0
124,202201,7.254880,1727120,1,0,0,0,1,0,0,0,1,0,0
128,202201,7.254880,1727120,1,0,0,0,1,0,0,1,0,0,0
132,202201,56.068785,1727120,1,0,0,0,1,0,0,0,0,0,1
136,202201,162.650825,1727120,1,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,202304,62.665842,3742,0,0,0,1,0,1,1,0,0,0,0
267,202304,62.665842,3742,0,0,0,1,0,1,0,0,1,0,0
271,202304,62.665842,3742,0,0,0,1,0,1,0,1,0,0,0
275,202304,469.296091,3742,0,0,0,1,0,1,0,0,0,0,1


In [157]:
dataParoPredic['Total'] = dataParoDummiesPredic['Total']

In [158]:
dataParoFinal = pd.concat([dataParo,dataParoPredic])
dataParoFinal = dataParoFinal.to_csv('DataParoFinal.csv')